In [7]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [8]:
# Set the paths for the train and validation datasets
train_path = ".\\dataset\\train"
valid_path = ".\\dataset\\validation"

In [9]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16


In [10]:
from PIL import Image
from tensorflow.keras.preprocessing import image
# Define the batch size and image size
batch_size = 32
img_size = (224, 224)

# Set up data generators for the train and validation datasets
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
train_generator = train_datagen.flow_from_directory(train_path, target_size=img_size, batch_size=batch_size, class_mode='binary')

valid_datagen = ImageDataGenerator(rescale=1./255)
valid_generator = valid_datagen.flow_from_directory(valid_path, target_size=img_size, batch_size=batch_size, class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [11]:
# Load the pre-trained VGG16 model
vgg = VGG16(input_shape=[224,224,3], weights='imagenet', include_top=False)

# Freeze the weights of the VGG16 model
for layer in vgg.layers:
    layer.trainable = False

# Define the input layer for the model
input_layer = Input(shape=[224,224,3])

# Pass the input layer through the VGG16 model to get the features
vgg_output = vgg(input_layer)

# Flatten the output from the VGG16 model
flatten_layer = Flatten()(vgg_output)

# Add a fully connected layer with 64 units
fc1 = Dense(units=64, activation='relu')(flatten_layer)

# Add the output layer with 2 units and use the sigmoid activation function
output_layer = Dense(units=1, activation='sigmoid')(fc1)

In [12]:
model_path = '.\\modelTest.h5'

In [14]:
from tensorflow import keras

# Check if model file exists
if os.path.isfile(model_path):
    # Load existing model
    model = keras.models.load_model(model_path)
else:
    # Create the model
    model = Model(inputs=input_layer, outputs=output_layer)
    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    # Train the model
    model.fit(train_generator, epochs=10, validation_data=valid_generator)
    model.save(model_path)

In [16]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input

img_path = 'C:\\Users\\ATH\work\\test2.jpg'
img = image.load_img(img_path, target_size=(224, 224))

# Convert the input image to a numpy array
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)

# Use the model to predict the class probabilities for the input image
preds = model.predict(img_array)


In [17]:
# Get the predicted class label
if preds[0][0] == 1:
    label = 'A'
else:
    label = 'B'